# "Euphotic zone residence time of Antarctic Bottom Water"

### Code for additional sensitivity test of the influence of different particle subsets on Euphotic zone residence

Corresponding to Yinghuan Xie (yinghuan.xie@utas.edu.au)

In [8]:
# Define the threshld for euphotic zone PAR in (watts per square metres)
# 2  mol/m^2/day: 5.439369304008213
# 3  mol/m^2/day: 8.15905395601232
# 4  mol/m^2/day: 10.878738608016427
PARZ_thre = 8.159
PARZ_thre_L = 5.439
PARZ_thre_U = 10.879

In [1]:
# Importing the relevant modules. 
import numpy as np
import xarray as xr
import os
import re
from glob import glob
import netCDF4 as nc
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import cosima_cookbook as cc
session =cc.database.create_session()
expt = '01deg_jra55v13_ryf9091'
from os import sys
sys.path.append('/home/581/yx9454/PhD_Ch1')
from My_Py_Func import Ch1_defs as c1d

In [2]:
# In each experiment, we define 06-18, release-06, 05-release, 87-05,
# loop_start_point-87 (if availiable) as No 0,1,2,3,4(if availiable)

master_path = '/g/data/jk72/yx9454/runs/parcels/output_after_Aug/'
traj_input = {'MLS_ON':[{'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Following_MLS_ON/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/'},
                        {'out_freq': 0.25,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/'},
                        {'out_freq': 0.25,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Following_MLS_ON/'},
                        {'out_freq': 0.25,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Following_Loop_MLS_ON/'}],
}

In [3]:
Chl_a_ver = 'Chl-a_based_on_OCCCI_v6'
var_in_arrs_0,var_in_arrs_1,var_in_arrs_2,var_in_arrs_3,var_in_arrs_4 = {},{},{},{},{}

for exp_name in ['MLS_ON']:
    files = sorted(glob('/g/data/jk72/yx9454/PAR_from_iaf_cycle4/'+exp_name+'/'+Chl_a_ver+'/var_in_arrs_*.npz'))

    var_in_arrs_list = [var_in_arrs_0,var_in_arrs_1,var_in_arrs_2,var_in_arrs_3,var_in_arrs_4]
    for i in [0,1,2,3,4]:
        print(files[i])
        for pvar in ['PARS','PARZ','Pt_ind']:
            PVAR   = np.load(files[i])[pvar]
            var_in_arrs_list[i][exp_name+'_'+pvar]=PVAR
            print(pvar,' loaded')

/g/data/jk72/yx9454/PAR_from_iaf_cycle4/MLS_ON/Chl-a_based_on_OCCCI_v6/var_in_arrs_0_AABW_only_PARS_Z.npz
PARS  loaded
PARZ  loaded
Pt_ind  loaded
/g/data/jk72/yx9454/PAR_from_iaf_cycle4/MLS_ON/Chl-a_based_on_OCCCI_v6/var_in_arrs_1_AABW_only_PARS_Z.npz
PARS  loaded
PARZ  loaded
Pt_ind  loaded
/g/data/jk72/yx9454/PAR_from_iaf_cycle4/MLS_ON/Chl-a_based_on_OCCCI_v6/var_in_arrs_2_AABW_only_PARS_Z.npz
PARS  loaded
PARZ  loaded
Pt_ind  loaded
/g/data/jk72/yx9454/PAR_from_iaf_cycle4/MLS_ON/Chl-a_based_on_OCCCI_v6/var_in_arrs_3_AABW_only_PARS_Z.npz
PARS  loaded
PARZ  loaded
Pt_ind  loaded
/g/data/jk72/yx9454/PAR_from_iaf_cycle4/MLS_ON/Chl-a_based_on_OCCCI_v6/var_in_arrs_4_AABW_only_PARS_Z.npz
PARS  loaded
PARZ  loaded
Pt_ind  loaded


In [17]:
MLS_ON_stopsign_list   = np.load('/g/data/jk72/yx9454/PAR_from_iaf_cycle4/MLS_ON/stopsign_list_for_20yrs_150m_thre.npz')['stopsign_list']
CDW_inds_MLS_ON   = ~(np.isnan(MLS_ON_stopsign_list))

CDW_inds = CDW_inds_MLS_ON

In [5]:
total_shallower_than_EZD_days_back_only = {}
total_shallower_than_EZD_days           = {}

In [9]:
for exp_name in ['MLS_ON']:
    # Backwards following
    #total_seconds = (T1 - dt.datetime(1970,1,1)).total_seconds()
    ind_t = -1
    out_freq = traj_input[exp_name][3]['out_freq']
    following    = np.sum(var_in_arrs_3[exp_name+'_PARZ'][:,:ind_t]>PARZ_thre,axis=1)*out_freq

    # Backwards first year
    out_freq = traj_input[exp_name][2]['out_freq']
    first_yr = np.sum((var_in_arrs_2[exp_name+'_PARZ'][:,:]>PARZ_thre),axis=1)*out_freq

    #The loop exp, starts from 2018-12-1
    ind_t = -1

    out_freq = traj_input[exp_name][4]['out_freq']
    loop = np.sum((var_in_arrs_4[exp_name+'_PARZ'][:,:ind_t]>PARZ_thre),axis=1)*out_freq

    # Forwards first year
    if True:
        out_freq = traj_input[exp_name][0]['out_freq']
        following_forward = np.sum((var_in_arrs_0[exp_name+'_PARZ'][:,:]>PARZ_thre),axis=1)*out_freq

    # Forwards following
    if True:
        ind_t = -1
        out_freq = traj_input[exp_name][1]['out_freq']
        first_yr_forward = np.sum((var_in_arrs_1[exp_name+'_PARZ'][:,:ind_t]>PARZ_thre),axis=1)*out_freq

    total_shallower_than_EZD_days_back_only[exp_name] = following + first_yr + loop
    total_shallower_than_EZD_days[exp_name]           = following + first_yr + following_forward + first_yr_forward + loop

In [18]:
 AABW_shallower_than_EZD_plus_CDW =  (total_shallower_than_EZD_days[exp_name]>0) & (CDW_inds)

In [21]:
number = int(94123*0.7)
for n in np.arange(20):
    print('Test', n+1)
    # Define the range and length of the random sequence
    low = 0
    high = 94123
    
    # Create a unique sequence of numbers in the desired range
    unique_sequence = np.arange(low, high)
    
    # Shuffle the unique sequence to make it random
    np.random.shuffle(unique_sequence)
    
    # Select the first 70000 elements to get a random sequence of length 70000
    random_sequence = unique_sequence[:number]
    
    # Check the length of the generated sequence
    print('random choose ',len(random_sequence),' particles')
    
    # Sort the random sequence in ascending order
    sorted_sequence = np.sort(random_sequence)
    
    # Check the first few elements of the sorted sequence
    #print(sorted_sequence[-10:])
    
    # Build the random mask
    mask = np.ones(94123)
    mask[sorted_sequence]=0
    mask = mask==1
    
    
    mean1 = np.mean(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask])
    median1 = np.median(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask])
    p10 = np.percentile(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask],10)
    p90 = np.percentile(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask],90)

    print('Mean:',100*(base_mean-mean1)/base_mean,'%',
        'Median:',100*(base_median-median1)/base_median,'%',
        'P10:',100*(base_p10-p10)/base_p10,'%',
        'P90',100*(base_p90-p90)/base_p90,'%')

Test 1
random choose  65886  particles
Mean: -0.29091734459914564 % Median: -0.9900990099009901 % P10: 0.0 % P90 -0.10471204188495961 %
Test 2
random choose  65886  particles
Mean: -1.0756680428997276 % Median: -1.9801980198019802 % P10: 0.0 % P90 -1.3089005235602094 %
Test 3
random choose  65886  particles
Mean: 0.555020742628886 % Median: 0.9900990099009901 % P10: 0.0 % P90 1.3089005235602094 %
Test 4
random choose  65886  particles
Mean: -0.9820579907707533 % Median: 0.0 % P10: 0.0 % P90 -2.094240837696335 %
Test 5
random choose  65886  particles
Mean: 1.0240455615035122 % Median: 1.9801980198019802 % P10: 0.0 % P90 -0.5235602094240838 %
Test 6
random choose  65886  particles
Mean: 1.0097404038392668 % Median: 0.49504950495049505 % P10: 0.0 % P90 0.7853403141361257 %
Test 7
random choose  65886  particles
Mean: -0.8801286554511351 % Median: -2.9702970297029703 % P10: 0.0 % P90 -0.2617801047120419 %
Test 8
random choose  65886  particles
Mean: 0.5401308641915853 % Median: 0.990099009

In [67]:
number = int(94123*0.5)
for n in np.arange(20):
    print('Test', n+1)
    # Define the range and length of the random sequence
    low = 0
    high = 94123
    
    # Create a unique sequence of numbers in the desired range
    unique_sequence = np.arange(low, high)
    
    # Shuffle the unique sequence to make it random
    np.random.shuffle(unique_sequence)
    
    # Select the first 70000 elements to get a random sequence of length 70000
    random_sequence = unique_sequence[:number]
    
    # Check the length of the generated sequence
    print('random choose ',len(random_sequence),' particles')
    
    # Sort the random sequence in ascending order
    sorted_sequence = np.sort(random_sequence)
    
    # Check the first few elements of the sorted sequence
    #print(sorted_sequence[-10:])
    
    # Build the random mask
    mask = np.ones(94123)
    mask[sorted_sequence]=0
    mask = mask==1
    
    
    mean1 = np.mean(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask])
    median1 = np.median(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask])
    p10 = np.percentile(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask],10)
    p90 = np.percentile(total_shallower_than_EZD_days[exp_name][AABW_shallower_than_EZD_plus_CDW & mask],90)
    
    print('Mean:',100*(base_mean-mean1)/base_mean,'%',
        'Median:',100*(base_median-median1)/base_median,'%',
        'P10:',100*(base_p10-p10)/base_p10,'%',
        'P90',100*(base_p90-p90)/base_p90,'%')

Test 1
random choose  47061  particles
Mean: -0.02985682639033317 % Median: -0.9029345372460497 % P10: 0.0 % P90 0.048038430744624806 %
Test 2
random choose  47061  particles
Mean: 1.0173079179246154 % Median: 1.1286681715575622 % P10: 3.75 % P90 0.8807045636509208 %
Test 3
random choose  47061  particles
Mean: -0.3906646178244796 % Median: 0.6772009029345373 % P10: 0.0 % P90 -0.32025620496397117 %
Test 4
random choose  47061  particles
Mean: 0.7392081148462215 % Median: 1.580135440180587 % P10: 0.0 % P90 1.0408326661329064 %
Test 5
random choose  47061  particles
Mean: -1.2159400753797664 % Median: -1.580135440180587 % P10: -1.25 % P90 -0.8246597277823131 %
Test 6
random choose  47061  particles
Mean: 0.406043596902098 % Median: 0.6772009029345373 % P10: 0.0 % P90 0.2401921537229784 %
Test 7
random choose  47061  particles
Mean: -0.34848401781289806 % Median: -1.3544018058690745 % P10: -2.5 % P90 1.120896717373899 %
Test 8
random choose  47061  particles
Mean: -0.33393314265729385 % M